# Progetto BigData
## Valerio Di Zio e Francesco Magnani

intro here

In [1]:
%%configure -f
{"executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [2]:
val bucketname = "unibo-bd2324-fmagnani"

val path_book_sample = "s3a://"+bucketname+"/project/book_sample134.json"
val path_book_sample32row = "s3a://"+bucketname+"/project/book_sample32row.json"
val path_positive_words = "s3a://"+bucketname+"/project/positive-words.txt"
val path_negative_words = "s3a://"+bucketname+"/project/negative-words.txt"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1707834106787_0002,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2324-fmagnani
path_book_sample: String = s3a://unibo-bd2324-fmagnani/project/book_sample134.json
path_book_sample32row: String = s3a://unibo-bd2324-fmagnani/project/book_sample32row.json
path_positive_words: String = s3a://unibo-bd2324-fmagnani/project/positive-words.txt
path_negative_words: String = s3a://unibo-bd2324-fmagnani/project/negative-words.txt
res3: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1707834106787_0002/


In [3]:
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.DataFrame
import org.apache.spark.broadcast.Broadcast

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.rdd.RDD
import org.apache.spark.sql.DataFrame
import org.apache.spark.broadcast.Broadcast


In [4]:
case class BookReview(
    id: String,
    overall:Double,
    reviewTime:String,
    reviewerID:String,
    asin:String,
    reviewText:String,
    summary:String
)

object BookReview{
    def extract(row:org.apache.spark.sql.Row) = {
        val overall = row.getAs[Double]("overall")
        val reviewTime = row.getAs[String]("reviewTime")
        val reviewerID = row.getAs[String]("reviewerID")
        val asin = row.getAs[String]("asin")
        val reviewText = row.getAs[String]("reviewText")
        val summary = row.getAs[String]("summary")
        val id = reviewerID + "-" + asin
    
        new BookReview(id, overall, reviewTime, reviewerID, asin, reviewText, summary)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class BookReview
defined object BookReview
Companions must be defined together; you may wish to use :paste mode for this.


In [5]:
val bookSampleRdd32row = spark.read.option("mode", "DROPMALFORMED").json(path_book_sample).map(BookReview.extract)
//val bookSampleRdd32row = spark.read.option("mode", "DROPMALFORMED").json(path_book_sample32row).map(BookReview.extract)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bookSampleRdd32row: org.apache.spark.sql.Dataset[BookReview] = [id: string, overall: double ... 5 more fields]


In [6]:
val positiveWords = sc.textFile(path_positive_words)
val negativeWords = sc.textFile(path_negative_words)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

positiveWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-fmagnani/project/positive-words.txt MapPartitionsRDD[5] at textFile at <console>:27
negativeWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-fmagnani/project/negative-words.txt MapPartitionsRDD[7] at textFile at <console>:27


In [7]:
def splitAndClean(id:String, text: String): Array[(String, String, Int)] = 
    text.split("\\s+").map(el => (id, el.toLowerCase.replaceAll("[^a-zA-Z\\s]", ""), 1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitAndClean: (id: String, text: String)Array[(String, String, Int)]


# Tentativi di ottimizzazione
1. utilizzo di variabili broadcast per le liste di parole uguali per tutti
2. ripartizione dei dataset per ottimizzare il join

# Prima fase del Job 1

Recensioni divise in parole tramite flat map, dopodiché join con parole positive e negative.

In [40]:
def countOccurrenciesOfWordReview(reviews: DataFrame): DataFrame = 
    reviews.
        groupBy("id", "word").
        agg(sum("occurrencies").alias("occurrencies"))


def joinAndMapWords(review:DataFrame, wordsDataset: Broadcast[Array[String]]): DataFrame = {
    val wordDF = review.sparkSession.createDataFrame(wordsDataset.value.map(Tuple1.apply)).toDF("word")
    
    val aggregRevDF = countOccurrenciesOfWordReview(review)
    
    return aggregRevDF.
        join(wordDF, aggregRevDF("word") === wordDF("word")).
        groupBy("id").
        agg(sum("occurrencies").alias("occurrencies"))
}

    

val reviewsDataset = bookSampleRdd32row.
    select("id", "reviewText").
    dropDuplicates(Seq("id")).
    filter(review => review.getAs[String]("reviewText") != null && review.getAs[String]("reviewText").length > 3).
    flatMap(review => splitAndClean(review.getAs[String]("id"), review.getAs[String]("reviewText"))).
    filter(row => row._2.length > 3).
    toDF("id", "word", "occurrencies").
    cache()

val positiveWordsBroadcast = sc.broadcast(positiveWords.collect())
val negativeWordsBroadcast = sc.broadcast(negativeWords.collect())


val withPositiveDF = joinAndMapWords(reviewsDataset, positiveWordsBroadcast)
    
val withNegativeDF = joinAndMapWords(reviewsDataset, negativeWordsBroadcast)


val reducedResult = withPositiveDF.
    join(withNegativeDF, Seq("id"), "outer").
    toDF("id", "valuePositive", "valueNegative").
    withColumn("estimated_overall", coalesce(col("valuePositive"), lit(0L)) - coalesce(col("valueNegative"), lit(0L)))
     

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

countOccurrenciesOfWordReview: (reviews: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
joinAndMapWords: (review: org.apache.spark.sql.DataFrame, wordsDataset: org.apache.spark.broadcast.Broadcast[Array[String]])org.apache.spark.sql.DataFrame
reviewsDataset: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, word: string ... 1 more field]
positiveWordsBroadcast: org.apache.spark.broadcast.Broadcast[Array[String]] = Broadcast(99)
negativeWordsBroadcast: org.apache.spark.broadcast.Broadcast[Array[String]] = Broadcast(101)
withPositiveDF: org.apache.spark.sql.DataFrame = [id: string, occurrencies: bigint]
withNegativeDF: org.apache.spark.sql.DataFrame = [id: string, occurrencies: bigint]
reducedResult: org.apache.spark.sql.DataFrame = [id: string, valuePositive: bigint ... 2 more fields]


# Seconda fase del Job 1 - Calcolo risultati

Qui calcoliamo i risultati utilizzando una normalizzazione Z-Score, successivamente normalizzata a sua volta tramite Min-Max tra 1 e 5, in modo da confrontarli con i voti delle recensioni stesse.

In [41]:
val resultDataframe = reducedResult.select("id", "estimated_overall").cache()

val minValue = resultDataframe.agg(min("estimated_overall")).head().getLong(0).toDouble
val maxValue = resultDataframe.agg(max("estimated_overall")).head().getLong(0).toDouble


// Calculate mean and standard deviation

val meanAndStdDF = resultDataframe.agg(mean("estimated_overall"), stddev("estimated_overall")).collect()
val meanValue = meanAndStdDF(0)(0).asInstanceOf[Double]
val stdValue = meanAndStdDF(0)(1).asInstanceOf[Double]

// Normalize the data using Z-score normalization formula

val normalizedDF = resultDataframe.withColumn("z_score", (col("estimated_overall") - meanValue) / stdValue)

// Scale the normalized values to be between -1 and 1

val minMaxDF = normalizedDF.agg(min("z_score"), max("z_score")).collect()
val minZScore = minMaxDF(0)(0).asInstanceOf[Double]
val maxZScore = minMaxDF(0)(1).asInstanceOf[Double]


val normalizedAndScaledDF = normalizedDF.withColumn("estimated_overall", 
                                                    (col("z_score") - minZScore) / (maxZScore - minZScore) * 4 + 1).
    select("id", "estimated_overall")

// DEPRECATED
// a = min, b = max, c = 1 e d = 5
def mapIntToOverallVote(toMap: Double, fromMin: Double, fromMax: Double, toMin:Double=1, toMax:Double=5): Double =
     toMin + ((toMax-toMin)/(fromMax-fromMin))*(toMap-fromMin)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

resultDataframe: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, estimated_overall: bigint]
minValue: Double = -82.0
maxValue: Double = 91.0
meanAndStdDF: Array[org.apache.spark.sql.Row] = Array([1.8817995696529966,3.5709445103782658])
meanValue: Double = 1.8817995696529966
stdValue: Double = 3.5709445103782658
normalizedDF: org.apache.spark.sql.DataFrame = [id: string, estimated_overall: bigint ... 1 more field]
minMaxDF: Array[org.apache.spark.sql.Row] = Array([-23.490087657723784,24.956478649091867])
minZScore: Double = -23.490087657723784
maxZScore: Double = 24.956478649091867
normalizedAndScaledDF: org.apache.spark.sql.DataFrame = [id: string, estimated_overall: double]
mapIntToOverallVote: (toMap: Double, fromMin: Double, fromMax: Double, toMin: Double, toMax: Double)Double


In [45]:
val relativeErrors = 
bookSampleRdd32row.
    map(review => (review.id, review.overall)).
    toDF("id", "real_value").
    join(normalizedAndScaledDF, Seq("id")).
    withColumn("relative_error", abs(coalesce(col("real_value"), lit(3.0)) - 
                                     coalesce(col("estimated_overall"), lit(3.0))) / 
                                   coalesce(col("real_value"), lit(3.0)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

relativeErrors: org.apache.spark.sql.DataFrame = [id: string, real_value: double ... 2 more fields]


In [46]:
val sumOfRelativeErrors = relativeErrors.select("relative_error").agg(sum("relative_error")).
                                collect()(0).getDouble(0)
val meanError =  sumOfRelativeErrors / relativeErrors.count

println("Errore Medio: " + meanError*100 + "%")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sumOfRelativeErrors: Double = 74703.89026974584
meanError: Double = 0.4240178581671454
Errore Medio: 42.401785816714536%
